### Training


In [1]:
%load_ext tensorboard

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import librosa
import math
import random


2023-12-18 19:22:51.910516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 19:22:53.425182: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
DATA_PATH = "data.json"

VAL_SIZE = 0.2 # percentage of dataset
TEST_SIZE = 0.1 # percentage of dataset


In [3]:
# load training data from json file

with open(DATA_PATH, "r") as f:
    data = json.load(f)

X = np.array(data["mfcc"])
y = np.array(data["labels"])


In [4]:
# split data into train, validation and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=VAL_SIZE)

# add an axis to input sets to match the shape CNN expects (last axis is like channel in color images)
X_train = X_train[..., np.newaxis]
X_test = X_test[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]


In [5]:
# choose hyperparameters to tune

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )


2023-12-18 19:23:50.877065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 19:23:51.040508: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 19:23:51.041106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 19:23:51.042715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 19:23:51.043202: I tensorflow/compile

In [6]:
def train_test_model(hparams):
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(keras.layers.Dropout(hparams[HP_DROPOUT]))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer=hparams[HP_OPTIMIZER],
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    fit_log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=fit_log_dir, histogram_freq=1)

    model.fit(x=X_train,
              y=y_train,
              validation_data=(X_validation, y_validation),
              batch_size=32,
              epochs=30,
              callbacks=[tensorboard_callback])
    
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy


def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy)

In [7]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
Epoch 1/30


2023-12-18 19:24:00.539052: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2023-12-18 19:24:00.831160: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-18 19:24:01.170637: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb4d0001840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 19:24:01.170720: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 970, Compute Capability 5.2
2023-12-18 19:24:01.206595: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-18 19:24:01.585497: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-18 19:24:01.676418: I ./tensorflow/compiler/jit/device_compi

212/212 [==============================] - 8s 17ms/step - loss: 1.7213 - accuracy: 0.3921 - val_loss: 1.4956 - val_accuracy: 0.4498
Epoch 2/30
212/212 [==============================] - 3s 15ms/step - loss: 1.1864 - accuracy: 0.5853 - val_loss: 1.5532 - val_accuracy: 0.4639
Epoch 3/30
212/212 [==============================] - 3s 15ms/step - loss: 0.9605 - accuracy: 0.6612 - val_loss: 1.6599 - val_accuracy: 0.4858
Epoch 4/30
212/212 [==============================] - 3s 14ms/step - loss: 0.8189 - accuracy: 0.7097 - val_loss: 1.1079 - val_accuracy: 0.6401
Epoch 5/30
212/212 [==============================] - 3s 14ms/step - loss: 0.7118 - accuracy: 0.7552 - val_loss: 1.0396 - val_accuracy: 0.6578
Epoch 6/30
212/212 [==============================] - 3s 15ms/step - loss: 0.6149 - accuracy: 0.7862 - val_loss: 0.8295 - val_accuracy: 0.7317
Epoch 7/30
212/212 [==============================] - 3s 14ms/step - loss: 0.5637 - accuracy: 0.8038 - val_loss: 0.9300 - val_accuracy: 0.7128
Epoch 8/30

In [8]:
%tensorboard --logdir logs/hparam_tuning

In [10]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2023-12-18 19:43:25.938132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 19:43:26.702147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-18 19:43:27.539293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 19:43:27.563768: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-18 19:4